In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import sys
from utils import *
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from dataset_processed import Graph_Bert_Dataset_fine_tune
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

h:\MeTDDI_final\codeocean\Classification\Unseen_DDIs\mol_graph.py:2: UserWarning: ignore
  warnings.warn('ignore')


In [2]:
from model_motif_level import * 
from model_atom_level import *

In [3]:
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf:
    print(module.__name__, module.__version__)

2.5.0
sys.version_info(major=3, minor=8, micro=0, releaselevel='final', serial=0)
matplotlib 3.6.2
numpy 1.23.4
pandas 1.5.2
sklearn 1.2.0
tensorflow 2.5.0


In [4]:
def input_solver1(sample,sample1,sample2,sample3,sample4,sample5,\
    sample6,sample7,sample8,sample9,sample10,sample11,sample12,sample13,sample14,sample15,sample16):
    return {'molecule_sequence1': sample,'molecule_sequence2': sample1, 'adj_matrix1': sample2,
           'adj_matrix2': sample3,'dist_matrix1': sample4,'dist_matrix2': sample5,
           'atom_features1':sample6,'atom_features2':sample7,'adjoin_matrix1_atom':sample8,
           'adjoin_matrix2_atom':sample9,'dist_matrix1_atom':sample10,'dist_matrix2_atom':sample11,
           'atom_match_matrix1':sample12,'atom_match_matrix2':sample13,'sum_atoms1':sample14,'sum_atoms2':sample15}, sample16

In [5]:
dataFolder = '/data/Classification/UnseenDDIs'
tr_dataset = pd.read_csv(dataFolder + '/tr_dataset.csv')
val_dataset = pd.read_csv(dataFolder + '/val_dataset.csv')
tst_dataset = pd.read_csv(dataFolder + '/tst_dataset.csv')

In [6]:
tokenizer = Mol_Tokenizer('/code/Classification/UnseenDDIs/token_id.json')
map_dict = np.load('/code/Classification/UnseenDDIs/preprocessed_drug_info.npy',allow_pickle=True).item()

In [7]:
train_dataset_,validation_dataset, test_dataset_ = Graph_Bert_Dataset_fine_tune(tr_dataset,val_dataset,tst_dataset,label_field='DDI',tokenizer=tokenizer,map_dict=map_dict,batch_size = 64).get_data()
train_dataset = train_dataset_.map(input_solver1)
val_dataset_ = validation_dataset.map(input_solver1) 
test_dataset = test_dataset_.map(input_solver1)

In [8]:
param = {'name': 'Small', 'num_layers': 4, 'num_heads': 8, 'd_model': 256}
arch = param   ## small 3 4 128   medium: 6 6  256     large:  12 8 516
num_layers = arch['num_layers']
num_heads =  arch['num_heads']
d_model =  arch['d_model']*2 
dff = d_model 
input_vocab_size = tokenizer.get_vocab_size
dropout_rate = 0.1
training = False 

In [9]:
## motif_level inputs
motif_inputs = Input(shape=(None,), name = "molecule_sequence")
motif_adj_inputs = Input(shape=(None,None), name= "adj_matrix") 
motif_dist_inputs = Input(shape=(None,None), name= "dist_matrix")
### atom_level inputs
atom_inputs = Input(shape=(None,61), name = "atom_features")
atom_adj_inputs = Input(shape=(None,None), name= "atom_adj_matrix") 
atom_dist_inputs = Input(shape=(None,None), name= "atom_dist_matrix")
atom_match_matrix = Input(shape=(None,None), name= "atom_match_matrix")
sum_atoms = Input(shape=(None,None), name= "sum_atoms")

In [10]:
## build atom_level model
Outseq_atom,*_,encoder_padding_mask_atom = EncoderModel_atom(num_layers = 2,d_model = arch['d_model'],dff = dff,\
                                                             num_heads = num_heads)(atom_inputs,adjoin_matrix = atom_adj_inputs,\
    dist_matrix = atom_dist_inputs,atom_match_matrix = atom_match_matrix,sum_atoms = sum_atoms,training = training) 
## build motif_level model
Outseq_motif,*_,encoder_padding_mask_motif = EncoderModel_motif(num_layers = num_layers,d_model = d_model,dff = dff,\
                                            num_heads = num_heads,input_vocab_size = input_vocab_size)(motif_inputs,adjoin_matrix = motif_adj_inputs,\
    dist_matrix = motif_dist_inputs,atom_level_features = Outseq_atom,training = training)
model_motif = Model(inputs = [atom_inputs,atom_adj_inputs,atom_dist_inputs,
            atom_match_matrix,sum_atoms,motif_inputs,motif_adj_inputs,motif_dist_inputs], outputs = [Outseq_motif,encoder_padding_mask_motif,Outseq_atom])

In [11]:
### Build dual inputs
### motif-level inputs
motif_inputs1 = Input(shape=(None,), name= "molecule_sequence1")
motif_inputs2 = Input(shape=(None,), name= "molecule_sequence2")
# mask_inputs1 = create_padding_mask(inputs1)
# mask_inputs2 = create_padding_mask(inputs2)
motif_adj_inputs1 = Input(shape=(None,None), name= "adj_matrix1") 
motif_adj_inputs2 = Input(shape=(None,None), name= "adj_matrix2")
motif_dist_inputs1 = Input(shape=(None,None), name= "dist_matrix1")
motif_dist_inputs2 = Input(shape=(None,None), name= "dist_matrix2")
### atom level inputs
atom_inputs1 = Input(shape=(None,61), name = "atom_features1")
atom_inputs2 = Input(shape=(None,61), name = "atom_features2")
atom_adj_inputs1 = Input(shape=(None,None), name= "adjoin_matrix1_atom") 
atom_adj_inputs2 = Input(shape=(None,None), name= "adjoin_matrix2_atom") 
atom_dist_inputs1 = Input(shape=(None,None), name= "dist_matrix1_atom")
atom_dist_inputs2 = Input(shape=(None,None), name= "dist_matrix2_atom")
atom_match_matrix1 = Input(shape=(None,None), name= "atom_match_matrix1")
atom_match_matrix2 = Input(shape=(None,None), name= "atom_match_matrix2")
sum_atoms1 = Input(shape=(None,None), name= "sum_atoms1")
sum_atoms2 = Input(shape=(None,None), name= "sum_atoms2")

In [12]:
# build weight sharing model
druga_trans,encoder_padding_mask_a,Outseq_atoma = model_motif([atom_inputs1,atom_adj_inputs1,atom_dist_inputs1\
    ,atom_match_matrix1,sum_atoms1,motif_inputs1,motif_adj_inputs1,motif_dist_inputs1]) 
drugb_trans,encoder_padding_mask_b,Outseq_atomb = model_motif([atom_inputs2,atom_adj_inputs2,atom_dist_inputs2\
    ,atom_match_matrix2,sum_atoms2,motif_inputs2,motif_adj_inputs2,motif_dist_inputs2]) 

In [13]:
# build co-attention layers and Fcls
Co_attention_layers = Co_Attention_Layer(d_model,k = 128,name = 'Co_attention_layer')
fc1 = tf.keras.layers.Dense(d_model/2, activation='relu') 
dropout1 = tf.keras.layers.Dropout(dropout_rate) 
fc2 = tf.keras.layers.Dense(d_model/4, activation='relu') 
dropout2 = tf.keras.layers.Dropout(dropout_rate) 
fc3 = tf.keras.layers.Dense(4,activation='softmax')

In [14]:
### To avoid high similarity scores
Wa = tf.keras.layers.Dense(d_model) 
Wb = tf.keras.layers.Dense(d_model)

# %%
druga_trans_,drugb_trans_,*_ = Co_attention_layers([Wa(druga_trans),Wb(drugb_trans)])
output1_2 = tf.keras.layers.Concatenate()([druga_trans_,drugb_trans_])
output1_2 = fc1(output1_2)
output1_2 = dropout1(output1_2,training=training) 
output1_2 = fc2(output1_2)
output1_2 = dropout2(output1_2,training=training) 
output1_2 = fc3(output1_2)

In [15]:
# build MeTDDI
models = Model(inputs=[atom_inputs1,atom_adj_inputs1,atom_dist_inputs1\
    ,atom_match_matrix1,sum_atoms1,motif_inputs1,motif_adj_inputs1,motif_dist_inputs1,
    atom_inputs2,atom_adj_inputs2,atom_dist_inputs2\
    ,atom_match_matrix2,sum_atoms2,motif_inputs2,motif_adj_inputs2,motif_dist_inputs2],outputs =[output1_2])

In [16]:
models.load_weights('/code/Classification/UnseenDDIs/Saved_weights/weights_unseenDDIs.h5')

In [17]:
from sklearn import metrics

In [18]:
def evaluation(preds,truths,average = 'micro',ndigits = 3):
    pred_res_to_labels = np.argmax(preds,axis=1)
    label_to_onehot = np.eye(4)[truths]
    acc = round(metrics.accuracy_score(truths,pred_res_to_labels),3)
    auc = round(metrics.roc_auc_score(label_to_onehot,preds,average = average),ndigits)
    aupr = round(metrics.average_precision_score(label_to_onehot,preds,average = average),ndigits) 
    print('ACC:',acc,'AUROC:',auc,'AUPR:',aupr) 

In [19]:
pred_res = models.predict(val_dataset_,verbose=1) 
labels = val_dataset['DDI'].tolist() 

536/536 [==============================] - 50s 85ms/step


In [20]:
evaluation(pred_res,labels)

ACC: 0.924 AUROC: 0.993 AUPR: 0.98
